# 1. Data preprocessing - DC and NY

In this notebook, we preprocess the data for washingtondc and newyorkcity, the procedure of which includes the following:


In [1]:
# Load necessary packages 
library(tidyverse)
library(yaml)
library(rnoaa)
library(mice)

configs <- read_yaml("./_config.yaml") 
comp_data_dir <- configs$competition_data   # competition data
data_dir <- configs$data_dir                # output data

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.1     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘mice’


The following object is masked from ‘package:stats’:

    filter


The following objects are masked from ‘package:base’:

    cbind, rbind




## 1a. Clean the data

Remove rows with:

* Phenophase_Status unknown (-1)

* AGDD == 0

In [2]:
usa_npn <- read.csv(paste0(comp_data_dir, "/USA-NPN_status_intensity_observations_data.csv")) %>%
    dplyr::select(Longitude, Latitude, Elevation_in_Meters, State, Species, Observation_Date, Day_of_Year, Phenophase_Status, AGDD, Accum_Prcp, Species_ID) %>%
    filter(Phenophase_Status != -1) %>%
    filter(AGDD > 0) %>%
    mutate(date = as.Date(Observation_Date, format = "%m/%d/%y")) %>%
    mutate(year = year(date)) %>%
    mutate(month = month(date)) %>%
    mutate(day = day(date)) %>%
    # dplyr::select(-Observation_Date) %>%
    filter(month %in% c(3,4,5))
head(usa_npn)

,Longitude,Latitude,Elevation_in_Meters,State,Species,Observation_Date,Day_of_Year,Phenophase_Status,AGDD,Accum_Prcp,Species_ID,date,year,month,day
,<dbl>,<dbl>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<int>,<date>,<dbl>,<dbl>,<int>
1,-122.8555,45.4856,63,OR,emarginata,3/4/10,63,1,508.50,274,87,2010-03-04,2010,3,4
2,-122.8555,45.4856,63,OR,emarginata,3/14/10,73,1,582.75,320,87,2010-03-14,2010,3,14
3,-122.8555,45.4856,63,OR,emarginata,3/15/10,74,1,594.00,320,87,2010-03-15,2010,3,15
4,-122.8555,45.4856,63,OR,emarginata,3/27/10,86,0,716.75,347,87,2010-03-27,2010,3,27
5,-122.8555,45.4856,63,OR,emarginata,4/4/10,94,0,776.25,428,87,2010-04-04,2010,4,4
6,-122.8555,45.4856,63,OR,emarginata,4/18/10,108,0,920.25,464,87,2010-04-18,2010,4,18


## 1b. Process DC data

* Pull weather data upto the most recent available date from NOAA

* Compute the accumulated growing degree days as instructed in the datafield descriptions.

In [3]:
# Define functions to pull (imputed if missing) temperature data
F01_get_temperature <- function(stationid, date_min = "1950-01-01", date_max = "2023-05-31") {

    dat <- ghcnd_search(stationid = stationid, var = c("TMAX", "TMIN", "PRCP"), 
               date_min = date_min, date_max = date_max) %>%
               purrr::reduce(left_join, by = "date") %>%
               dplyr::select(id.x, date, tmax, tmin, prcp) %>%
               dplyr::rename_with(~ "id", id.x) %>%
               mutate(tmax = tmax/10) %>%      # in C
               mutate(tmin = tmin/10) %>%      # in C
               mutate(prcp = prcp/10) %>%      # in mm
               mutate(year = format(date, "%Y")) %>%
               mutate(month = as.integer(strftime(date, '%m'))) %>%
               mutate(day = as.integer(strftime(date, '%d')))
    
    return(dat)
}

F01_get_imp_temperature <- function(city_station_pair, date_min = "1950-01-01", date_max = "2023-05-31", imp_method = "pmm") {

    station_ids <- city_station_pair$id
    city_temp_list <- list()

    for (c in seq_len(length(station_ids))) {

        skip_to_next <- 0
        
        temp_df <- tryCatch(
            {F01_get_temperature(station_ids[c]
            , date_min = date_min
            , date_max = date_max)
            }
        , error = function(x) skip_to_next <<-1 )
        
        if (skip_to_next == 1 ){
            next
        }
        # Impute missing data
        # - check missing data
        n_missing <- sum(is.na(temp_df[, c("tmax", "tmin", "prcp")]))

        if (n_missing > 0) {
            tempData <- mice(temp_df, m = 3, method = imp_method)

            # complete set
            imputed_temp <- complete(tempData, 3)
        
        } else {
            imputed_temp <- temp_df
        }
        city_temp_list[[c]] <- imputed_temp
    }
    out <- city_temp_list %>% bind_rows()
    return(out)
}


In [4]:
# Pull weather data
# - washington dc
dc_temp_raw <- F01_get_imp_temperature(
    data.frame(city = "washingtondc", id = "USC00186350")
    , date_min = "1950-01-01"
    , date_max = "2024-05-01"
    )
# head(dc_temp_raw)
dc_min_year <- min(dc_temp_raw$year)

dc_blooms <- read.csv(paste0(comp_data_dir, "/washingtondc.csv")) %>%
    filter(year >= dc_min_year) %>%
    mutate(bloom_date = as.Date(bloom_date, format = "%Y-%m-%d"))
head(dc_blooms)

using cached file: ~/.cache/R/noaa_ghcnd/USC00186350.dly

date created (size, mb): 2023-02-06 22:23:38 (2.732)



file min/max dates: 1948-08-01 / 2022-11-30




 iter imp variable
  1   1  tmax  tmin  prcp
  1   2  tmax  tmin  prcp
  1   3  tmax  tmin  prcp
  2   1  tmax  tmin  prcp
  2   2  tmax  tmin  prcp
  2   3  tmax  tmin  prcp
  3   1  tmax  tmin  prcp
  3   2  tmax  tmin  prcp
  3   3  tmax  tmin  prcp
  4   1  tmax  tmin  prcp
  4   2  tmax  tmin  prcp
  4   3  tmax  tmin  prcp
  5   1  tmax  tmin  prcp
  5   2  tmax  tmin  prcp
  5   3  tmax  tmin  prcp


Warning message:
“Number of logged events: 2”


,location,lat,long,alt,year,bloom_date,bloom_doy
,<chr>,<dbl>,<dbl>,<int>,<int>,<date>,<int>
1,washingtondc,38.88535,-77.03863,0,1950,1950-04-09,99
2,washingtondc,38.88535,-77.03863,0,1951,1951-04-06,96
3,washingtondc,38.88535,-77.03863,0,1952,1952-04-09,100
4,washingtondc,38.88535,-77.03863,0,1953,1953-03-27,86
5,washingtondc,38.88535,-77.03863,0,1954,1954-04-06,96
6,washingtondc,38.88535,-77.03863,0,1955,1955-04-02,92


In [5]:
dc_lat <- usa_npn[usa_npn$State == "DC", "Latitude"][1]
dc_long <- usa_npn[usa_npn$State == "DC", "Longitude"][1]
dc_alt <- usa_npn[usa_npn$State == "DC", "Elevation_in_Meters"][1]

# Compute AGDD and process
dc_gdd <- dc_temp_raw %>%
    mutate(gdd = ifelse((tmax+tmin)/2 > 0, (tmax+tmin)/2, 0)) %>%
    group_by(year) %>%
    mutate(AGDD = cumsum(gdd)) %>%
    mutate(Accum_Prcp = cumsum(prcp)) %>%
    ungroup() %>%
    filter(month %in% c(3,4,5)) %>%
    data.frame(.) %>%
    mutate(State = "DC") %>%
    mutate(Species = "yedoensis") %>%
    mutate(Latitude = dc_lat) %>%
    mutate(Longitude = dc_long) %>%
    mutate(Elevation_in_Meters = dc_alt) %>%
    mutate(Day_of_Year = as.numeric(format(date, "%j"))) %>%
    mutate(date = as.Date(date, format = "%Y-%m-%d")) %>%
    merge(y = dc_blooms[,c("bloom_date", "bloom_doy")], by.x = "date", by.y = "bloom_date", how = "left", all.x = TRUE) %>%
    mutate(Phenophase_Status = ifelse(is.na(bloom_doy), 0, 1)) %>%
    dplyr::select(Longitude, Latitude, Elevation_in_Meters, State, Species, Day_of_Year, Phenophase_Status, AGDD, Accum_Prcp, date, year, month, day)
head(dc_gdd)

,Longitude,Latitude,Elevation_in_Meters,State,Species,Day_of_Year,Phenophase_Status,AGDD,Accum_Prcp,date,year,month,day
,<dbl>,<dbl>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<date>,<chr>,<int>,<int>
1,-77.07354,38.90951,40,DC,yedoensis,60,0,416.35,120.2,1950-03-01,1950,3,1
2,-77.07354,38.90951,40,DC,yedoensis,61,0,418.00,120.2,1950-03-02,1950,3,2
3,-77.07354,38.90951,40,DC,yedoensis,62,0,418.00,120.2,1950-03-03,1950,3,3
4,-77.07354,38.90951,40,DC,yedoensis,63,0,418.00,120.2,1950-03-04,1950,3,4
5,-77.07354,38.90951,40,DC,yedoensis,64,0,423.25,120.2,1950-03-05,1950,3,5
6,-77.07354,38.90951,40,DC,yedoensis,65,0,431.05,120.2,1950-03-06,1950,3,6


## 1c. Process NY data

* Following the same procedure as the above.

In [6]:
ny_blooms <- usa_npn %>%
    filter(State == "NY") %>%
    filter(Species_ID == 228) %>% 
    arrange(Observation_Date) %>% 
    # mutate(year = year(Observation_Date)) %>% 
    group_by(year) %>% 
    summarize(first_flower_index = min(which(Phenophase_Status == 1)),
                bloom_date = strftime(date[first_flower_index], format = '%Y-%m-%d'),
                bloom_doy = Day_of_Year[first_flower_index],
                .groups = 'drop') %>% 
    filter(!is.na(bloom_doy)) %>% 
    select(-first_flower_index) %>% 
    mutate(location = 'newyorkcity') %>%
    data.frame(.) %>%
    mutate(bloom_date = as.Date(bloom_date, format = "%Y-%m-%d")) %>%
    mutate(bloom_doy = as.numeric(format(bloom_date, "%j")))

ny_blooms[(nrow(ny_blooms)+1), ] <- c(2023, "2023-04-30", as.numeric(format(as.Date("2023-04-30"), "%j")), "newyorkcity")

ny_min_year <- min(ny_blooms$year)

ny_temp_raw <- F01_get_imp_temperature(
    data.frame(city = "newyorkcity", id = "USW00094728")
    , date_min = paste0(ny_min_year, "-01-01")
    , date_max = "2024-05-01"
    )
head(ny_temp_raw)


Warning message:
“There were 2 warnings in `summarize()`.
The first warning was:
ℹ In argument: `first_flower_index = min(which(Phenophase_Status == 1))`.
ℹ In group 1: `year = 2011`.
Caused by warning in `min()`:
! no non-missing arguments to min; returning Inf
ℹ Run `dplyr::last_dplyr_warnings()` to see the 1 remaining warning.”
using cached file: ~/.cache/R/noaa_ghcnd/USW00094728.dly

date created (size, mb): 2024-02-27 22:38:05 (8.578)



file min/max dates: 1869-01-01 / 2024-02-29




 iter imp variable
  1   1  tmax  tmin  prcp
  1   2  tmax  tmin  prcp
  1   3  tmax  tmin  prcp
  2   1  tmax  tmin  prcp
  2   2  tmax  tmin  prcp
  2   3  tmax  tmin  prcp
  3   1  tmax  tmin  prcp
  3   2  tmax  tmin  prcp
  3   3  tmax  tmin  prcp
  4   1  tmax  tmin  prcp
  4   2  tmax  tmin  prcp
  4   3  tmax  tmin  prcp
  5   1  tmax  tmin  prcp
  5   2  tmax  tmin  prcp
  5   3  tmax  tmin  prcp


Warning message:
“Number of logged events: 2”


,id,date,tmax,tmin,prcp,year,month,day
,<chr>,<date>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<int>
1,USW00094728,2014-01-01,0.6,-4.3,0.0,2014,1,1
2,USW00094728,2014-01-02,0.6,-7.7,8.4,2014,1,2
3,USW00094728,2014-01-03,-7.7,-12.7,7.4,2014,1,3
4,USW00094728,2014-01-04,-1.6,-13.2,0.0,2014,1,4
5,USW00094728,2014-01-05,4.4,-2.7,3.6,2014,1,5
6,USW00094728,2014-01-06,12.8,-7.1,9.1,2014,1,6


In [7]:
ny_lat <- usa_npn[usa_npn$State == "NY", "Latitude"][1]
ny_long <- usa_npn[usa_npn$State == "NY", "Longitude"][1]
ny_alt <- usa_npn[usa_npn$State == "NY", "Elevation_in_Meters"][1]

ny_gdd <- ny_temp_raw %>% 
    # filter(year == 2023) %>%
    mutate(gdd = ifelse((tmax+tmin)/2 > 0, (tmax+tmin)/2, 0)) %>%
    group_by(year) %>%
    mutate(AGDD = cumsum(gdd)) %>%
    mutate(Accum_Prcp = cumsum(prcp)) %>%
    ungroup() %>%
    filter(month %in% c(3,4,5)) %>%
    data.frame(.) %>%
    mutate(State = "NY") %>%
    mutate(Species = "yedoensis") %>%
    mutate(Latitude = ny_lat) %>%
    mutate(Longitude = ny_long) %>%
    mutate(Elevation_in_Meters = ny_alt) %>%
    mutate(Day_of_Year = as.numeric(format(date, "%j"))) %>%
    mutate(date = as.Date(date, format = "%Y-%m-%d")) %>%
    merge(y = ny_blooms[,c("bloom_date", "bloom_doy")], by.x = "date", by.y = "bloom_date", how = "left", all.x = TRUE) %>%
    mutate(Phenophase_Status = ifelse(is.na(bloom_doy), 0, 1)) %>%
    dplyr::select(Longitude, Latitude, Elevation_in_Meters, State, Species, Day_of_Year, Phenophase_Status, AGDD, Accum_Prcp, date, year, month, day)
head(ny_gdd)

,Longitude,Latitude,Elevation_in_Meters,State,Species,Day_of_Year,Phenophase_Status,AGDD,Accum_Prcp,date,year,month,day
,<dbl>,<dbl>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<date>,<chr>,<int>,<int>
1,-74.02538,41.09339,122,NY,yedoensis,60,0,109.40,210.2,2014-03-01,2014,3,1
2,-74.02538,41.09339,122,NY,yedoensis,61,0,111.60,210.2,2014-03-02,2014,3,2
3,-74.02538,41.09339,122,NY,yedoensis,62,0,111.60,211.2,2014-03-03,2014,3,3
4,-74.02538,41.09339,122,NY,yedoensis,63,0,111.60,211.2,2014-03-04,2014,3,4
5,-74.02538,41.09339,122,NY,yedoensis,64,0,111.95,211.2,2014-03-05,2014,3,5
6,-74.02538,41.09339,122,NY,yedoensis,65,0,111.95,211.2,2014-03-06,2014,3,6


## 1e. Process BC data

In [8]:
bc_blooms <- read.csv(paste0(comp_data_dir, "/vancouver.csv")) %>%
    mutate(bloom_date = as.Date(bloom_date, format = "%Y-%m-%d"))
bc_lat <- bc_blooms[1, "lat"]
bc_long <- bc_blooms[1, "long"]
bc_alt <- bc_blooms[1, "alt"]

bc_temp_raw <- F01_get_imp_temperature(
    data.frame(city = "vancouver", id = "CA001108395")
    , date_min = "2022-01-01"
    , date_max = "2024-05-01"
    )

bc_gdd <- bc_temp_raw %>%
    mutate(gdd = ifelse((tmax+tmin)/2 > 0, (tmax+tmin)/2, 0)) %>%
    group_by(year) %>%
    mutate(AGDD = cumsum(gdd)) %>%
    mutate(Accum_Prcp = cumsum(prcp)) %>%
    ungroup() %>%
    filter(month %in% c(3,4,5)) %>%
    data.frame(.) %>%
    mutate(State = "BC") %>%
    mutate(Species = "yedoensis") %>%
    mutate(Latitude = bc_lat) %>%
    mutate(Longitude = bc_long) %>%
    mutate(Elevation_in_Meters = bc_alt) %>%
    mutate(Day_of_Year = as.numeric(format(date, "%j"))) %>%
    mutate(date = as.Date(date, format = "%Y-%m-%d")) %>%
    merge(y = bc_blooms[,c("bloom_date", "bloom_doy")], by.x = "date", by.y = "bloom_date", how = "left", all.x = TRUE) %>%
    mutate(Phenophase_Status = ifelse(is.na(bloom_doy), 0, 1)) %>%
    dplyr::select(Longitude, Latitude, Elevation_in_Meters, State, Species, Day_of_Year, Phenophase_Status, AGDD, Accum_Prcp, date, year, month, day)

using cached file: ~/.cache/R/noaa_ghcnd/CA001108395.dly

date created (size, mb): 2023-02-06 22:23:48 (1.325)



file min/max dates: 1957-01-01 / 2023-02-28




 iter imp variable
  1   1  tmax  tmin  prcp
  1   2  tmax  tmin  prcp
  1   3  tmax  tmin  prcp
  2   1  tmax  tmin  prcp
  2   2  tmax  tmin  prcp
  2   3  tmax  tmin  prcp
  3   1  tmax  tmin  prcp
  3   2  tmax  tmin  prcp
  3   3  tmax  tmin  prcp
  4   1  tmax  tmin  prcp
  4   2  tmax  tmin  prcp
  4   3  tmax  tmin  prcp
  5   1  tmax  tmin  prcp
  5   2  tmax  tmin  prcp
  5   3  tmax  tmin  prcp


Warning message:
“Number of logged events: 2”


## 1d. Merge and save the data

In [9]:
## Merge all data
usa_npn_out <- usa_npn %>%
    dplyr::select(Longitude, Latitude, Elevation_in_Meters, State, Species, Day_of_Year, Phenophase_Status, AGDD, Accum_Prcp, date, year, month, day) %>%
    filter(!(State %in% c("DC", "NY", "BC"))) %>%
    rbind(., dc_gdd) %>%
    rbind(., ny_gdd) %>%
    rbind(., bc_gdd)
tail(usa_npn_out)

write.csv(usa_npn_out, paste0(data_dir, "/A31_america_temperatures2.csv"), row.names = FALSE)
print("done")

,Longitude,Latitude,Elevation_in_Meters,State,Species,Day_of_Year,Phenophase_Status,AGDD,Accum_Prcp,date,year,month,day
,<dbl>,<dbl>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<date>,<chr>,<dbl>,<int>
8518,-123.1636,49.2237,24,BC,yedoensis,146,0,1127.35,501.3,2022-05-26,2022,5,26
8519,-123.1636,49.2237,24,BC,yedoensis,147,0,1140.20,509.3,2022-05-27,2022,5,27
8520,-123.1636,49.2237,24,BC,yedoensis,148,0,1152.00,509.3,2022-05-28,2022,5,28
8521,-123.1636,49.2237,24,BC,yedoensis,149,0,1165.35,509.3,2022-05-29,2022,5,29
8522,-123.1636,49.2237,24,BC,yedoensis,150,0,1177.35,514.4,2022-05-30,2022,5,30
8523,-123.1636,49.2237,24,BC,yedoensis,151,0,1191.75,514.4,2022-05-31,2022,5,31


[1] "done"
